In [1]:
import requests
from pymongo import MongoClient
from pymongo import GEOSPHERE
import pandas as pd
import json
import os
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from getpass import getpass
import time
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from collections import defaultdict
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

### MongoDB query ###

In [2]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
companies = db.get_collection("companies")

In [4]:
query = {"category_code": "games_video", "number_of_employees": {"$gte": 100, "$lte": 300}, "offices.country_code": {"$exists": True}, "offices.latitude": {"$exists": True}, "offices.latitude": {"$exists": True}}
projection = {"_id":0, "name":1, "category_code": 1, "offices.latitude": 1, "offices.longitude": 1, "number_of_employees": 1}
results = list(companies.find(query, projection))

In [5]:
# checking how many game companies, with the number of employes from 100 to 300, per country

query = {"category_code": "games_video", "number_of_employees": {"$gte": 100, "$lte": 300}, "offices.country_code": {"$exists": True}, "offices.latitude": {"$exists": True}, "offices.latitude": {"$exists": True}}
projection = {"_id": 0, "offices.country_code": 1}

results = list(companies.find(query, projection))
country_count = defaultdict(int)

for result in results:
    country_code = result.get("offices", [{}])[0].get("country_code", "Unknown")
    country_count[country_code] += 1

for country, count in country_count.items():
    print(f"{country} - {count}")

USA - 21
SGP - 1
GBR - 2
IND - 2
ITA - 1
NLD - 1
CHE - 1
DEU - 1
SWE - 1
FRA - 1
UKR - 1


In [6]:
# As USA is the country with more game companies, checkong now the cities

query = {
    "category_code": "games_video",
    "number_of_employees": {"$gte": 100, "$lte": 300},
    "offices.country_code": "USA",
    "offices.city": {"$exists": True},
    "offices.latitude": {"$exists": True},
    "offices.longitude": {"$exists": True}
}

projection = {
    "_id": 0,
    "offices.city": 1
}

results = list(companies.find(query, projection))
us_city_count = defaultdict(int)

for result in results:
    city = result.get("offices", [{}])[0].get("city", "Unknown")
    us_city_count[city] += 1

print("City, Count")
for city, count in us_city_count.items():
    print(f"{city} - {count}")

City, Count
New York - 3
San Francisco - 7
Redwood City - 1
Bellevue - 1
Los Angeles - 3
Dundee - 1
Westwood - 1
Brooklyn - 1
Torino - 1
Sunnyvale - 3
Seattle - 1
Oxford - 1
Kharkov - 1


In [13]:
# looking for game companies, with number of employees Between 100 and 300

my_map = folium.Map(location=[40.726155, -121.997355], zoom_start=3)

for index, row in df.iterrows():
    if row['category_code'] == 'games_video':
        marker_color = 'red'
        marker_size = 'large'

    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        icon=folium.Icon(color=marker_color, icon_color='white', icon=marker_size)
    ).add_to(my_map)
my_map

In [8]:
query = {"category_code": "games_video", "number_of_employees": {"$gte": 100, "$lte": 300}, "offices.country_code": {"$exists": True}, "offices.latitude": {"$exists": True}, "offices.latitude": {"$exists": True}}
projection = {"_id":0, "name":1, "category_code": 1, "offices.latitude": 1, "offices.longitude": 1, "number_of_employees": 1}
results2 = list(companies.find(query, projection))

In [10]:
df = pd.DataFrame(results2)
df.sample()

,name,category_code,number_of_employees,offices
22,RGB Networks,games_video,125,"[{'latitude': 37.4123706, 'longitude': -122.02..."


In [11]:
df['latitude'] = df['offices'].apply(lambda x: x[0]['latitude'])
df['longitude'] = df['offices'].apply(lambda x: x[0]['longitude'])
df.sample()

,name,category_code,number_of_employees,offices,latitude,longitude
27,ZillionTV,games_video,100,"[{'latitude': 37.3738725, 'longitude': -121.99...",37.373872,-121.997355


In [12]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.sample()

,name,category_code,number_of_employees,offices,latitude,longitude
1,Ustream,games_video,250,"[{'latitude': 37.392936, 'longitude': -122.079...",37.392936,-122.07948


In [14]:
token = getpass()

········


In [ ]:
client_id = getpass()

In [ ]:
client_secret = getpass()

### Getting nearby venues through Foursquare for San Fracisco ### 

In [ ]:
where = "San Francisco, USA"

In [15]:
lat = 37.765158
lon = -122.404234

In [16]:
# locating a random company in San Francisco from the game and number of employees criteria

latitude = 37.765158
longitude = -122.404234

m = folium.Map(location=[latitude, longitude], zoom_start=12)
folium.Marker([latitude, longitude], popup="Your Location").add_to(m)
m

In [17]:
# checking for close Child School

def requests_for_foursquare (query, lat, lon, radius=2000, limit=1):
    lat = 37.765158
    lon = -122.404234 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [18]:
child_school = requests_for_foursquare ("Preschool", lat, lon, radius=2000, limit=5)

In [19]:
near_child_school = []
for venue in child_school["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    near_child_school.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

child_school_df = pd.DataFrame(near_child_school)

child_school_df

,Name,Latitude,Longitude,Distance
0,Petite Explorer Preschool,37.761362,-122.417410,1229
1,Apple Tree Preschool,37.777038,-122.405075,1313
2,Potrero Kids Daniel Webster Preschool,37.760587,-122.395863,893
3,Studio Montessori,37.771043,-122.404865,658
4,Good Samaritan Preschool,37.751894,-122.406603,1488


In [20]:
# Creating a map centered around a location of child school 
m = folium.Map(location=[child_school_df.iloc[0]['Latitude'], child_school_df.iloc[0]['Longitude']], zoom_start=12)

for index, row in child_school_df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [37.765158, -122.404234]
folium.Marker(company_location, 
              tooltip='Zynga',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [21]:
# checking for close Starbucks

def requests_for_foursquare (query, lat, lon, radius=2000, limit=1):
    lat = 37.765158
    lon = -122.404234 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [22]:
starbucks = requests_for_foursquare ("Starbucks", lat, lon, radius=1000, limit=5)

In [23]:
near_starbucks = []
for venue in starbucks["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    near_starbucks.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

starbucks_df = pd.DataFrame(near_starbucks)

starbucks_df

,Name,Latitude,Longitude,Distance
0,Starbucks,37.765796,-122.403409,105
1,Starbucks,37.766862,-122.409564,493
2,Starbucks,37.767121,-122.393738,935
3,Starbucks,37.762898,-122.410549,600
4,Starbucks,37.770294,-122.403435,575


In [24]:
# Creating a map centered around a location of Starbucks
m = folium.Map(location=[starbucks_df.iloc[0]['Latitude'], starbucks_df.iloc[0]['Longitude']], zoom_start=12)

for index, row in starbucks_df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [37.765158, -122.404234]
folium.Marker(company_location, 
              tooltip='Zynga',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [25]:
# checking for close Vegan Restaurants

def requests_for_foursquare (query, lat, lon, radius=1000, limit=1):
    lat = 37.765158
    lon = -122.404234 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [26]:
vegan_restaurant = requests_for_foursquare ("Vegan", lat, lon, radius=1000, limit=5)

In [27]:
vegan = []
for venue in vegan_restaurant["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    vegan.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

vegan_restaurant_df = pd.DataFrame(vegan)

vegan_restaurant_df

,Name,Latitude,Longitude,Distance
0,Whole Foods,37.764381,-122.402680,167
1,Stable Cafe,37.763254,-122.415300,992
2,SOMA StrEat Food Park,37.769745,-122.412028,827
3,Goat Hill Pizza,37.762371,-122.397760,652
4,Atlas Cafe,37.758973,-122.411464,934


In [28]:
# Creating a map centered around a location of vegan restaurant
m = folium.Map(location=[vegan_restaurant_df.iloc[0]['Latitude'], vegan_restaurant_df.iloc[0]['Longitude']], zoom_start=13)

for index, row in vegan_restaurant_df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [37.765158, -122.404234]
folium.Marker(company_location, 
              tooltip='Zynga',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [29]:
# checking for close Basketball Stadium 


def requests_for_foursquare (query, lat, lon, radius=1000, limit=1):
    lat = 37.765158
    lon = -122.404234 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [30]:
basketball_stadium = requests_for_foursquare ("basketball", lat, lon, radius=1000, limit=5)

In [31]:
basketball = []
for venue in basketball_stadium["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    basketball.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

basketball_stadium_df = pd.DataFrame(basketball)

basketball_stadium_df

,Name,Latitude,Longitude,Distance
0,Berry Basketball Courts,37.772080,-122.397686,932
1,NBA Nation,37.759168,-122.412335,987
2,Jackson Playground,37.763975,-122.399001,420
3,Mission Creek Park,37.770963,-122.398624,1245
4,SOMA StrEat Food Park,37.769745,-122.412028,827


In [32]:
# Creating a map centered around a location of basketball stadium
m = folium.Map(location=[basketball_stadium_df.iloc[0]['Latitude'], basketball_stadium_df.iloc[0]['Longitude']], zoom_start=12)

for index, row in basketball_stadium_df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [37.765158, -122.404234]
folium.Marker(company_location, 
              tooltip='Zynga',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [33]:
# checking for close Pet Store 


def requests_for_foursquare (query, lat, lon, radius=1000, limit=1):
    lat = 37.765158
    lon = -122.404234 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [34]:
pet_shop = requests_for_foursquare ("Pet Store", lat, lon, radius=1000, limit=5)

In [35]:
pet_shop_ = []
for venue in pet_shop["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    pet_shop_.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

pet_shop_df = pd.DataFrame(pet_shop_)

pet_shop_df

,Name,Latitude,Longitude,Distance
0,Pawtrero Hill BathHouse & Feed,37.764155,-122.394496,862
1,Petco,37.766509,-122.410332,540
2,George,37.761277,-122.410811,721
3,Bernie's Grooming,37.766882,-122.412719,796
4,Mission: Cats,37.762355,-122.414243,922


In [36]:
# Creating a map centered around a location of Pet shop
m = folium.Map(location=[pet_shop_df.iloc[0]['Latitude'], pet_shop_df.iloc[0]['Longitude']], zoom_start=12)

for index, row in pet_shop_df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [37.765158, -122.404234]
folium.Marker(company_location, 
              tooltip='Zynga',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [37]:
# checking for close Airport 

def requests_for_foursquare (query, lat, lon, radius=500, limit=1):
    lat = 37.765158
    lon = -122.404234 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [38]:
airport = requests_for_foursquare ("International Airport", lat, lon, radius=30000) 

In [39]:
airport_ = []
for venue in airport["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    airport_.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

airport_df = pd.DataFrame(airport_)

airport_df

,Name,Latitude,Longitude,Distance
0,San Francisco International Airport (SFO),37.61649,-122.38681,16559


In [40]:
# Creating a map centered around a location of an Internation Airport

latitude = 37.765158
longitude = -122.404234

geolocator = Nominatim(user_agent="airport_finder")
airport_coordinates = (37.6213129, -122.3811443)  # San Francisco International Airport (SFO)
company_coordinates = (latitude, longitude)

m = folium.Map(location=company_coordinates, zoom_start=10)

folium.Marker(airport_coordinates, icon=folium.Icon(color='blue'), popup="San Francisco International Airport (SFO)").add_to(m)
folium.Marker(company_coordinates, icon=folium.Icon(color='green'), popup="Company Location").add_to(m)

m

In [41]:
# checking the location from the closest airport from San Fracisco - USA

latitude = 37.765158
longitude = -122.404234

geolocator = Nominatim(user_agent="airport_finder")
airport_coordinates = (37.6213129, -122.3811443)  # San Francisco International Airport (SFO)
distance = geodesic((latitude, longitude), airport_coordinates).miles
print(f"The closest airport is San Francisco International Airport (SFO), which is approximately {distance:.2f} miles away.")

The closest airport is San Francisco International Airport (SFO), which is approximately 10.00 miles away.


### Getting nearby venues through Foursquare for Los Angeles ### 

In [42]:
where_2 = "Los Angeles, USA"
latitude2 = 34.031276
longitude2 = -118.456806

In [43]:
# locating a random company in Los Angeles from the game and number of employees criteria
latitude2 = 34.031276
longitude2 = -118.456806

m = folium.Map(location=[latitude2, longitude2], zoom_start=10)

icon = folium.Icon(icon="cloud")
folium.Marker([latitude2, longitude2], popup="Your Location", icon=icon).add_to(m)

m

In [44]:
# checking for close Child School

def requests_for_foursquare (query, lat, lon, radius=1000, limit=1):
    lat = 34.031276
    lon = -118.456806 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [45]:
child_school_la = requests_for_foursquare ("Preschool", lat, lon, radius=1000, limit=5)

In [46]:
near_child_school_la = []
for venue in child_school_la["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    near_child_school_la.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

child_school_df2 = pd.DataFrame(near_child_school_la)

child_school_df2

,Name,Latitude,Longitude,Distance
0,Our World Preschool,34.036621,-118.457464,597
1,Little Village Nursery School,34.030477,-118.448647,744
2,Mindful Heart Kids,34.036955,-118.454715,659
3,Maple Tree,34.029973,-118.448566,772
4,Sunny Childcare Preschool,34.032045,-118.447335,876


In [47]:
# Creating a map centered around a location of child school 
m = folium.Map(location=[child_school_df2.iloc[0]['Latitude'], child_school_df2.iloc[0]['Longitude']], zoom_start=12)

for index, row in child_school_df2.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [34.031276, -118.456806]
folium.Marker(company_location, 
              tooltip='hulu',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [48]:
# checking for close Starbucks

def requests_for_foursquare (query, lat, lon, radius=2000, limit=1):
    lat = 34.031276 
    lon = -118.456806 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [49]:
starbucks_la = requests_for_foursquare ("Starbucks", lat, lon, radius=1000, limit=5)

In [50]:
near_starbucks_la = []
for venue in starbucks_la["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    near_starbucks_la.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

starbucks_df_la = pd.DataFrame(near_starbucks_la)

starbucks_df_la

,Name,Latitude,Longitude,Distance
0,Starbucks,34.024394,-118.460891,859
1,Starbucks,34.034989,-118.449162,812


In [51]:
# Creating a map centered around a location of Starbucks
m = folium.Map(location=[starbucks_df_la.iloc[0]['Latitude'], starbucks_df_la.iloc[0]['Longitude']], zoom_start=12)

for index, row in starbucks_df_la.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [34.031276, -118.456806]
folium.Marker(company_location, 
              tooltip='hulu',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [52]:
# checking for close Vegan Restaurants

def requests_for_foursquare (query, lat, lon, radius=1000, limit=1):
    lat = 34.031276 
    lon = -118.456806  
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [53]:
vegan_restaurant_la = requests_for_foursquare ("Vegan", lat, lon, radius=1000, limit=5)

In [54]:
vegan_la = []
for venue in vegan_restaurant_la["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    vegan_la.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

vegan_restaurant_df_la = pd.DataFrame(vegan_la)

vegan_restaurant_df_la

,Name,Latitude,Longitude,Distance
0,Bondi Harvest,34.031330,-118.462657,549
1,Trader Joe's,34.026369,-118.455772,556
2,Habayit,34.030399,-118.448928,722
3,Viet Noodle Bar,34.026691,-118.456203,514
4,Sweetgreen,34.034839,-118.458764,435


In [55]:
# Creating a map centered around a location of vegan restaurant
m = folium.Map(location=[vegan_restaurant_df_la.iloc[0]['Latitude'], vegan_restaurant_df_la.iloc[0]['Longitude']], zoom_start=12)

for index, row in vegan_restaurant_df_la.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [34.031276, -118.456806]
folium.Marker(company_location, 
              tooltip='hulu',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [56]:
# checking for close Airport 

def requests_for_foursquare (query, lat, lon, radius=500, limit=1):
    lat = 34.031276
    lon = -118.456806 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [57]:
airport_la = requests_for_foursquare ("International Airport", lat, lon, radius=50000, limit=5)

In [58]:
airport_la_ = []
for venue in airport_la["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    airport_la_.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

airport_df_la = pd.DataFrame(airport_la_)

airport_df_la

,Name,Latitude,Longitude,Distance
0,Los Angeles International Airport (LAX),33.94352,-118.408420,10798
1,Tom Bradley International Terminal (TBIT),33.94352,-118.408537,10686


In [59]:
# Creating a map centered around a location of vegan restaurant
m = folium.Map(location=[airport_df_la.iloc[0]['Latitude'], airport_df_la.iloc[0]['Longitude']], zoom_start=12)

for index, row in airport_df_la.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [34.031276, -118.456806]
folium.Marker(company_location, 
              tooltip='hulu',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [60]:
# checking the location from the closest airport from Los Angeles - USA

latitude = 34.031276
longitude = -118.456806

geolocator = Nominatim(user_agent="airport_finder")
airport_coordinates = (33.9439667, -118.4109258) 
distance = geodesic((latitude, longitude), airport_coordinates).miles
print(f"The closest airport is Tom Bradley International, Los Angeles, which is approximately {distance:.2f} miles away.")

The closest airport is Tom Bradley International, Los Angeles, which is approximately 6.57 miles away.


In [ ]:
"""
While the aiport is closer to the office in Los Angeles, more criteria were met for the office in San Fracisco
Deciding to place the office iin San Francisco, taking Zynga's place!


"""